In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [13]:
data = pd.read_excel("productdata.xlsx")
data.head()

,PRODUCTID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCTTYPEID,ProductLength
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650.0,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755.0,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537.0,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996.0,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112.0,598.424000


In [14]:
#check each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3847 entries, 0 to 3846
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PRODUCTID      3847 non-null   int64  
 1   TITLE          3847 non-null   object 
 2   BULLET_POINTS  2256 non-null   object 
 3   DESCRIPTION    1703 non-null   object 
 4   PRODUCTTYPEID  3669 non-null   float64
 5   ProductLength  3669 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 180.5+ KB


from the info above i can infer that the `bullet_points`, `description` , `peoducttypeid` and `productlength` all have missing data

In [15]:
data.describe()

,PRODUCTID,PRODUCTTYPEID,ProductLength
count,3.847000e+03,3669.000000,3669.000000
mean,1.456557e+06,3932.736986,1150.529020
std,8.666684e+05,3970.908660,2665.897894
min,1.303000e+03,0.000000,1.000000
25%,6.922785e+05,154.000000,507.873000
50%,1.441218e+06,2879.000000,640.000000
75%,2.214798e+06,6337.000000,1023.622046
max,2.999397e+06,13330.000000,96000.000000


In [16]:
#check for the number of unique product Id in the dataset
print(data.PRODUCTTYPEID.nunique())
print(data.PRODUCTID.nunique())

1278
3541


In [17]:
#check for missing data
data.isna().sum()

PRODUCTID           0
TITLE               0
BULLET_POINTS    1591
DESCRIPTION      2144
PRODUCTTYPEID     178
ProductLength     178
dtype: int64

In [18]:
#fill the categorical missing columns with "unknown"
data["BULLET_POINTS"].fillna("Unknown", inplace=True)
data["DESCRIPTION"].fillna("Unknown", inplace=True)
data["PRODUCTTYPEID"].fillna(data['PRODUCTTYPEID'].median(), inplace=True)
data["ProductLength"].fillna(data['ProductLength'].mean(), inplace=True)
data.isna().sum()

PRODUCTID        0
TITLE            0
BULLET_POINTS    0
DESCRIPTION      0
PRODUCTTYPEID    0
ProductLength    0
dtype: int64

In [19]:
#check for duplicates
print(data.duplicated().sum())
data.drop_duplicates(inplace=True)

217


In [20]:
#rename the coulmns
data.rename(columns={"PRODUCTID":"PRODUCT_ID", "PRODUCTTYPEID":"PRODUCT_TYPE_ID", "ProductLength":"PRODUCT_LENGTH"}, inplace=True)
data.columns

Index(['PRODUCT_ID', 'TITLE', 'BULLET_POINTS', 'DESCRIPTION',
       'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH'],
      dtype='object')

In [21]:
#checking for anomalies in the data
cols_to_check = ['PRODUCT_ID','PRODUCT_TYPE_ID', 'PRODUCT_LENGTH']
anomalies = {col: data[data[col] <0].sum() for col in cols_to_check}
print(anomalies)

{'PRODUCT_ID': PRODUCT_ID           0
TITLE                0
BULLET_POINTS        0
DESCRIPTION          0
PRODUCT_TYPE_ID    0.0
PRODUCT_LENGTH     0.0
dtype: object, 'PRODUCT_TYPE_ID': PRODUCT_ID           0
TITLE                0
BULLET_POINTS        0
DESCRIPTION          0
PRODUCT_TYPE_ID    0.0
PRODUCT_LENGTH     0.0
dtype: object, 'PRODUCT_LENGTH': PRODUCT_ID           0
TITLE                0
BULLET_POINTS        0
DESCRIPTION          0
PRODUCT_TYPE_ID    0.0
PRODUCT_LENGTH     0.0
dtype: object}


In [23]:
#using NLP for SEO optimization
import spacy
nlp = spacy.load("en_core_web_sm")

In [31]:
import re
#function to extract names entities and limit to 50 characters
def exxtract(text, max_length= 50):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents] #extract named entities
    # Extract key nouns (e.g., product type, category) if NER is missing entities
    nouns = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"] and len(token.text) > 2]
     # Combine entities and nouns for a better short title
    keywords = entities + nouns
   # Remove unwanted words (e.g., "set of", "includes", "features", special characters)
    keywords = [word for word in keywords if word.lower() not in ["set", "includes", "features", "pcs"]]
    keywords = [re.sub(r'[^\w\s-]', '', word) for word in keywords]  # Remove special characters

    # Generate short title
    short_title = " ".join(keywords) if keywords else text  # Use original if nothing found
    
    #truncate to max_length while keeping the words intact
    if len(short_title) > max_length:
        short_title = short_title[:max_length].rsplit(" ", 1)[0] #avoid cutting off words
    return short_title

In [32]:
data["SHORT_TITLE"] = data["TITLE"].apply(lambda x: exxtract(x, max_length=50))
data.head(5)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,SHORT_TITLE
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,Unknown,1650.0,2125.980000,ArtzFolio Canvas Fabric 5 feet 60 inch 2
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",Unknown,2755.0,393.700000,Marks Spencer Girls Pyjama Sets T86_2561C_Navy
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537.0,748.031495,PRIKNIK Horn Red Electric Air Horn Compressor
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996.0,787.401574,ALISHAH Womens Cotton Ankle Length Leggings Combo
4,283658,The United Empire Loyalists: A Chronicle of th...,Unknown,Unknown,6112.0,598.424000,The United Empire the Great Migration United


In [26]:
data.tail(5)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,SHORT_TITLE
3651,2919319,Generic Chiffon printed dupatta with Golden do...,Unknown,Unknown,2879.0,1150.52902,Generic Chiffon Hangings for Girls and Women
3652,90582,CAUGHT IN THE ACT (Loveswept),Unknown,Unknown,2879.0,1150.52902,CAUGHT IN THE ACT (Loveswept)
3653,2894923,GlobalNicheÂ® Leather Car Key Case Cover for f...,Unknown,Unknown,2879.0,1150.52902,Car Key Case Cover Fiesta Escort Key Ring Car
3654,1008258,"The Mountain Bigfoot Adult T-Shirt, Brown, Small",Unknown,Unknown,2879.0,1150.52902,"The Mountain Bigfoot Adult T-Shirt, Brown, Small"
3655,402457,Tales of Cydonia: Volume Two: Cydonia Labyrint...,Unknown,Unknown,2879.0,1150.52902,Cydonia Two Cydonia 2
